import numpy as np

%run "C:\Users\tomas\Downloads\OA_project_20221894\base\dataset_generation.ipynb" 

In [27]:
def check_constraints(ind):
    """
    Check if the given individual satisfies the constraints of our problem.

    Args:
        ind (list): A list representing an individual solution, where each element represents a location.

    Returns:
        bool: True if all constraints are satisfied, False otherwise.
    """

    size = len(ind)

    # All sessions must begin and end at D (0)
    if ind[0] != 0 or ind[-1] != 0:
        return False

    # If we skipped KS (6), we had to had gone directly from QS (3) to DV (8), and only skip after that sequence
    elif '-' in ind and ( ind.index(3) != (ind.index(8) - 1) or ind.index('-') < ind.index(8) or 6 in ind):
        return False

    # We cannot go from QG (4) directly to CS (5)

    elif ind.index(4) == ( ind.index(5) - 1 ):
        return False

    # The RG (7) have to be in the last half

    elif ind.index(7) <= size // 2  :
        return False

    else:
        return True

In [28]:
def generate_solution(ind_size=11):
    """
    Generates a random valid solution for our problem.

    Args:
        ind_size (int): The fixed size of each solution..

    Returns:
        list: A list representing a valid solution where each element represents a location.
    """

    trips = list( range(1, ind_size-1) )  # Possible places to go
    np.random.shuffle(trips)   # Shuffle the possible places to go, to get a random combination of them

    # Enforcing in the initialization that every session will start and end in D (0)
    sol =  [0] + trips + [0]

    # While we don't get a solution that don't violate any constraint, we generate another from scratch
    while not check_constraints(sol):
        sol = generate_solution()

    return sol

In [29]:
sample_sol = generate_solution()
sample_sol

[0, 3, 2, 8, 4, 6, 9, 7, 5, 1, 0]

In [30]:
def get_fitness(matrix):
    """
    Calculate the fitness of a solution based on a matrix of Geo earnings.

    Args:
        matrix (list): A matrix of Geo earnings.

    Returns:
        function: A function that calculates the fitness of a solution, ensuring it satisfies the specified constraints.
    """

    def get_fitness_inner(sol):
        """
        Calculate the fitness of a solution based on a given individual.

        Args:
            sol (list): An individual solution representing a sequence of areas.

        Returns:
            int: The fitness value of the solution.
        """

        if not check_constraints(sol):    # If the solution violates any of the constraints, it is assigned a very uncompetitive fitness
            return -1

        fitness = 0
        # Iterating from the second to the last element
        for i in range(1, len(sol)):
            if sol[i] != '-':        # If the current area was not skipped

                if sol[i-1] != '-':                         # And if the past area was not skipped
                    geo = matrix[sol[i-1]][sol[i]]              # We sum the geo of the trip to our fitness
                    if isinstance(geo, int):
                        fitness += geo

                else:                                       # And the past area was skipped
                    geo = matrix[sol[i-2]][sol[i]]              # We sum the geo of the trip (from the area before the
                                                                # skip to the area after it) to our fitness
                    if isinstance(geo, int):
                        fitness += geo
            else:
                continue              # If the current area was skipped, we skip this iteration since we don't have Geo to account for

        return fitness

    return get_fitness_inner

In [31]:
get_fitness(sample_matrix)(sample_sol)

153